In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./Comp7103_Assignment1_data_mining_and_NLP')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
!git clone https://github.com/google-research/bert.git
!pip install bert-serving-server # server
!pip install bert-serving-client # client, independent of 'bert-serving-server'
!pip install transformers

fatal: destination path 'bert' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=60138d0c78271cfd3a57526235ed90a4fd512ab18d1d4beaeadff88a3a678312
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built GPUtil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 M

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras import optimizers

import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

import random

In [ ]:
from transformers import GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification

In [ ]:
df = pd.read_csv("amazon/Reviews.csv")
df = df.dropna()
print(len(df))

568411


In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df = df.sample(n=int(len(df)*0.01))
print(len(df))

5684


In [ ]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

In [ ]:
from transformers import pipeline
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

In [ ]:
bert_df = df.loc[:,['Text','Score']]
bert_df['Score'] -= 1
train_df, test_df = train_test_split(bert_df, test_size=0.2, random_state=3074)
bert_df.head()

,Text,Score
422107,Happy to see one of my numerous cats favorite...,3
258259,They are made in Romania. NEED Salt in the wat...,2
495281,This Syrup is the real deal. No bogus flavors....,4
46835,This is probably one of the best BBQ sauces th...,4
422051,"Halo makes great products, my cats love them. ...",3


In [ ]:
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='gpt2', num_labels=5)
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='gpt2', config=model_config)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
gpt_df = df.loc[:,['Text','Score']]
gpt_df['Score'] -= 1
train_df, test_df = train_test_split(gpt_df, test_size=0.2, random_state=3074)
gpt_df.head()

,Text,Score
422107,Happy to see one of my numerous cats favorite...,3
258259,They are made in Romania. NEED Salt in the wat...,2
495281,This Syrup is the real deal. No bogus flavors....,4
46835,This is probably one of the best BBQ sauces th...,4
422051,"Halo makes great products, my cats love them. ...",3


In [ ]:
encoded_data_train = tokenizer(list(train_df['Text']), padding = 'longest', truncation=True)
encoded_data_test = tokenizer(list(test_df['Text']), padding = 'longest', truncation=True)

In [ ]:
input_ids_train = torch.tensor(np.array(encoded_data_train['input_ids']))
attention_masks_train = torch.tensor(np.array(encoded_data_train['attention_mask']))
labels_train = torch.tensor(train_df['Score'].to_numpy())

input_ids_test = torch.tensor(np.array(encoded_data_test['input_ids']))
attention_masks_test = torch.tensor(np.array(encoded_data_test['attention_mask']))
labels_test = torch.tensor(test_df['Score'].to_numpy())

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
batch_size = 4

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
# DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset
dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)        
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
# seed_val = 3074
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':     batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

f1_scores = []
training_loss = []
validation_loss = []    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        model = model.to(device)
        outputs = model(**inputs)
        # print(inputs['labels'])
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./bert_checkpopint/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    f1_scores.append(val_f1)
    validation_loss.append(val_loss)
    training_loss.append(loss_train_avg)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1137 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0529229772058475
Validation loss: 0.8305985359703763
F1 Score (Weighted): 0.6632330326139786


Epoch 2:   0%|          | 0/1137 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.754179257286013
Validation loss: 0.7807284634805431
F1 Score (Weighted): 0.6929459242766466


...


Epoch 8
Training loss: 0.4165240677163863
Validation loss: 1.3139088404723576
F1 Score (Weighted): 0.7147052968423231


Epoch 9:   0%|          | 0/1137 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.3779923692845753
Validation loss: 1.3452012555680357
F1 Score (Weighted): 0.7162733932345755


Epoch 10:   0%|          | 0/1137 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.3404035932775676
Validation loss: 1.3841120514375864
F1 Score (Weighted): 0.7188705658164761
